# I14Y Concepts to LINDAS Defined Term Sets

In [15]:
import sparqlmagic

# change endpoint globally for all %%sparql calls
sparqlmagic.sparql_endpoint = "https://test.ld.admin.ch/query"

## Show All Defined Term Sets

In [16]:
%%sparql
PREFIX schema: <http://schema.org/>
SELECT ?concept
WHERE {
  GRAPH <https://lindas.admin.ch/fso/i14y> {
    ?concept a schema:DefinedTermSet .
  }
}

,concept.type,concept.value
0,uri,https://register.ld.admin.ch/i14y/concept/BEWOHNERTYP
1,uri,https://register.ld.admin.ch/i14y/concept/BEWOHNERTYP/version/1.0.0
2,uri,https://register.ld.admin.ch/i14y/concept/DV_AUSLAENDERKATEGORIEN
3,uri,https://register.ld.admin.ch/i14y/concept/DV_AUSLAENDERKATEGORIEN/version/1.0.0
4,uri,https://register.ld.admin.ch/i14y/concept/DV_AUSWEISTYP
5,uri,https://register.ld.admin.ch/i14y/concept/DV_AUSWEISTYP/version/1.0.0
6,uri,https://register.ld.admin.ch/i14y/concept/DV_SEXE
7,uri,https://register.ld.admin.ch/i14y/concept/DV_SEXE/version/2.0.0
8,uri,https://register.ld.admin.ch/i14y/concept/ETAT_CIVIL
9,uri,https://register.ld.admin.ch/i14y/concept/ETAT_CIVIL/version/1.0.0


## Show Predicates and Objects

In [17]:
%%sparql df
PREFIX schema: <http://schema.org/>
SELECT ?p ?o
WHERE {
  <https://register.ld.admin.ch/i14y/concept/legalForm> ?p ?o .
}

,p.type,p.value,o.type,o.value,o.xml:lang,o.datatype
0,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,uri,http://schema.org/DefinedTermSet,nan,nan
1,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,uri,http://www.w3.org/2004/02/skos/core#ConceptScheme,nan,nan
2,uri,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,uri,https://version.link/Identity,nan,nan
3,uri,http://schema.org/name,literal,Rechtsform eines Unternehmens (identity),de,nan
4,uri,http://schema.org/name,literal,Legal form of a company (identity),en,nan
5,uri,http://schema.org/name,literal,Forme juridique d'une entreprise (identity),fr,nan
6,uri,http://schema.org/name,literal,Forma giuridica di una società (identity),it,nan
7,uri,http://schema.org/description,literal,"Die Codierung der Rechtsform ist vierstellig. Die ersten zwei Ziffern von «01» bis «05» erlauben eine übergeordnete Einteilung der Rechtsformen. Die Codierung «01» entspricht den Rechtsformen, die von den Handelsregistern angewendet werden. Die Codierung «02» gilt für Rechtsformen von öffentlich-rechtlichen Einheiten, welche nicht im Handelsregister eingeschrieben sind. Alle verbleibenden Rechtsformen ohne Handelsregister entsprechen der Codierung «03». Schliesslich wird die Codierung «04» für ausländische Einheiten und «05» für unbestimmt oder unbekannt verwendet. Anhand der letzten zwei Ziffern des vierstelligen Codes wird die explizite Rechtsform zugeordnet. Beispiel für «Rechtsform = 0106»: Über die ersten zwei Ziffern «01» wird ersichtlich, dass diese Rechtsform im schweizerischen Handelsregister angewendet wird. Die letzten zwei Ziffern «06» entsprechen der Rechtsform «Aktiengesellschaft».",de,nan
8,uri,http://schema.org/description,literal,"The coding of the legal form has four digits. The first two digits from ""01"" to ""05"" allow a higher-level classification of the legal forms. The code ""01"" corresponds to the legal forms applied by the commercial registers. The code ""02"" applies to legal forms of public-law entities that are not registered in the commercial register. All remaining legal forms without a commercial register correspond to the coding ""03"". Finally, the code ""04"" is used for foreign units and ""05"" for undetermined or unknown. The last two digits of the four-digit code are used to assign the specific legal form. Example for ""legal form = 0106"": The first two digits ""01"" indicate that this legal form is used in the Swiss commercial register. The last two digits ""06"" correspond to the legal form ""Corporation (Ltd)"".",en,nan
9,uri,http://schema.org/description,literal,"Le codage de la forme juridique est à quatre chiffres. Les deux premiers chiffres de « 01 » à « 05 » permettent une classification générale des formes juridiques. Le code « 01 » correspond aux formes juridiques utilisées par les registres du commerce. Le code « 02 » s'applique aux formes juridiques des entités de droit public qui ne sont pas inscrites au registre du commerce. Toutes les formes juridiques restantes sans registre du commerce correspondent à la codification « 03 ». Enfin, le code « 04 » est utilisé pour les entités étrangères et le « 05 » pour indéterminé ou inconnu. La forme juridique spécifique est attribuée à l'aide des deux derniers chiffres du code à quatre chiffres. Exemple pour « forme juridique = 0106 » : les deux premiers chiffres « 01 » indiquent que cette forme juridique est utilisée dans les registres du commerce suisses. Les deux derniers chiffres « 06 » correspondent à la forme juridique « société anonyme ».",fr,nan


## Start and Stop Dates of Versions

In [18]:
%%sparql
PREFIX schema: <http://schema.org/>
SELECT ?concept ?validFrom ?validThrough
WHERE {
  GRAPH <https://lindas.admin.ch/fso/i14y> {
    ?concept a schema:DefinedTermSet , <https://version.link/Version> .
    OPTIONAL {?concept schema:validFrom ?validFrom .}
    OPTIONAL {?concept schema:validThrough ?validThrough .}
  }
}

,concept.type,concept.value,validFrom.datatype,validFrom.type,validFrom.value
0,uri,https://register.ld.admin.ch/i14y/concept/BEWOHNERTYP/version/1.0.0,http://www.w3.org/2001/XMLSchema#date,literal,2020-09-13
1,uri,https://register.ld.admin.ch/i14y/concept/DV_AUSLAENDERKATEGORIEN/version/1.0.0,http://www.w3.org/2001/XMLSchema#date,literal,2009-12-31
2,uri,https://register.ld.admin.ch/i14y/concept/DV_AUSWEISTYP/version/1.0.0,http://www.w3.org/2001/XMLSchema#date,literal,2009-12-31
3,uri,https://register.ld.admin.ch/i14y/concept/DV_SEXE/version/2.0.0,http://www.w3.org/2001/XMLSchema#date,literal,2022-03-30
4,uri,https://register.ld.admin.ch/i14y/concept/ETAT_CIVIL/version/1.0.0,http://www.w3.org/2001/XMLSchema#date,literal,2021-07-06
5,uri,https://register.ld.admin.ch/i14y/concept/UMZUG/version/1.0.0,http://www.w3.org/2001/XMLSchema#date,literal,2011-12-31
6,uri,https://register.ld.admin.ch/i14y/concept/VOCAB_I14Y_ATTRIBUTION_ROLE/version/1.0.0,http://www.w3.org/2001/XMLSchema#date,literal,2023-06-14
7,uri,https://register.ld.admin.ch/i14y/concept/legalForm/version/1.1.0,http://www.w3.org/2001/XMLSchema#date,literal,2010-12-31
8,uri,https://register.ld.admin.ch/i14y/concept/VOCAB_EU_DATA_THEME/version/20220715.0.0,http://www.w3.org/2001/XMLSchema#date,literal,2022-07-15
9,uri,https://register.ld.admin.ch/i14y/concept/VOCAB_EU_FREQUENCY/version/20230614.0.0,http://www.w3.org/2001/XMLSchema#date,literal,2023-06-14


--> They all have a schema:validFrom but no schema:validThrough (even if they have a successor)

## Datacite Vocabulary

In [19]:
%%sparql df

SELECT DISTINCT ?p WHERE {
    GRAPH <https://lindas.admin.ch/fso/i14y> {
        ?s ?p ?o .
    }
    FILTER(CONTAINS(STR(?p), "datacite"))
}

,p.type,p.value
0,uri,https://datacite-metadata-schema.readthedocs.io/en/4.6/appendices/appendix-1/relationType/#haspart


In [20]:
df["p.value"][0]

'https://datacite-metadata-schema.readthedocs.io/en/4.6/appendices/appendix-1/relationType/#haspart'

## Version Link Vocabulary

In [21]:
%%sparql df

SELECT DISTINCT ?p WHERE {
    GRAPH <https://lindas.admin.ch/fso/i14y> {
        ?s ?p ?o .
    }
    FILTER(CONTAINS(STR(?p), "version.link"))
}

,p.type,p.value
0,uri,https://version.link/identity
1,uri,https://version.link/version
2,uri,https://version.link/successor
3,uri,https://version.link/predecessor


In [22]:
%%sparql df

SELECT (COUNT(?s) AS ?no) WHERE {
    GRAPH <https://lindas.admin.ch/fso/i14y> {
        ?s a <https://version.link/Identity> .
    }
}

,no.datatype,no.type,no.value
0,http://www.w3.org/2001/XMLSchema#integer,literal,59544


In [23]:
%%sparql df

SELECT (COUNT(?s) AS ?no) WHERE {
    GRAPH <https://lindas.admin.ch/fso/i14y> {
        ?s a <https://version.link/Version> .
    }
}

,no.datatype,no.type,no.value
0,http://www.w3.org/2001/XMLSchema#integer,literal,127425


## Additional Data on Terms (Annotations)

In [24]:
%%sparql df

PREFIX schema: <http://schema.org/>
SELECT * WHERE {
  <https://register.ld.admin.ch/i14y/concept/DV_KT_BEZ_GDE_SNAP/1> schema:hasPart ?part .
  ?part <https://www.w3.org/ns/oa#hasBody> ?body .
  ?body ?p ?o .
}

,part.type,part.value,body.type,body.value,p.type,p.value,o.type,o.value
0,bnode,N353869bd99954339a84edccd6d862a81,bnode,Ndf2691450c874130b4692f53a9d07ff0,uri,http://purl.org/dc/terms/type,literal,EXT_RESOURCE
1,bnode,N353869bd99954339a84edccd6d862a81,bnode,Ndf2691450c874130b4692f53a9d07ff0,uri,https://www.w3.org/ns/oa#hasTarget,uri,https://ld.admin.ch/municipality/version/13256
2,bnode,N5af748026fe84eb5bd5cbb6e898f0458,bnode,Nb1316f578aa349e48eb513d054c252d6,uri,http://schema.org/name,literal,1
3,bnode,N5af748026fe84eb5bd5cbb6e898f0458,bnode,Nb1316f578aa349e48eb513d054c252d6,uri,http://purl.org/dc/terms/title,literal,1
4,bnode,N5af748026fe84eb5bd5cbb6e898f0458,bnode,Nb1316f578aa349e48eb513d054c252d6,uri,http://purl.org/dc/terms/type,literal,ORDER
5,bnode,N986f696368fa4e07b559427d80539191,bnode,N85b0ae202d8b4b0f89cd04c1e644d3b6,uri,http://schema.org/name,literal,13256
6,bnode,N986f696368fa4e07b559427d80539191,bnode,N85b0ae202d8b4b0f89cd04c1e644d3b6,uri,http://purl.org/dc/terms/title,literal,13256
7,bnode,N986f696368fa4e07b559427d80539191,bnode,N85b0ae202d8b4b0f89cd04c1e644d3b6,uri,http://purl.org/dc/terms/type,literal,CODE_HISTORY
8,bnode,Nb497e595e9f4452cb325f6c30482c9bc,bnode,N0b079aaca2354676b14f55c312eb9959,uri,http://schema.org/name,literal,1
9,bnode,Nb497e595e9f4452cb325f6c30482c9bc,bnode,N0b079aaca2354676b14f55c312eb9959,uri,http://purl.org/dc/terms/title,literal,1


Questions:

- why so many duplications of information?
- where is the information for "HIER_LEVEL" (see: https://www.i14y.admin.ch/de/catalog/concepts/08dc23f0-d04d-2d2f-a9f5-9cea80695acf/content/8100/ZH/B_101/1)?

In [25]:
%%sparql df

SELECT * WHERE {
  <https://register.ld.admin.ch/i14y/concept/DV_KT_BEZ_GDE_SNAP/1> <http://www.w3.org/ns/shacl#property> ?shacl .
  ?shacl ?p ?o .
}

,shacl.type,shacl.value,p.type,p.value,o.type,o.value
0,bnode,anon-genid-37cbba4232c8401ab0faacd8da802561-15700,uri,http://www.w3.org/ns/shacl#path,uri,http://www.w3.org/2004/02/skos/core#notation
1,bnode,anon-genid-37cbba4232c8401ab0faacd8da802561-15700,uri,http://www.w3.org/ns/shacl#datatype,uri,http://www.w3.org/2001/XMLSchema#string
2,bnode,anon-genid-37cbba4232c8401ab0faacd8da802561-15701,uri,http://www.w3.org/ns/shacl#path,uri,http://www.w3.org/2004/02/skos/core#notation
3,bnode,anon-genid-37cbba4232c8401ab0faacd8da802561-15701,uri,http://www.w3.org/ns/shacl#datatype,uri,http://www.w3.org/2001/XMLSchema#string
4,bnode,anon-genid-b5f6f4f8800e435c9e169bc3b47dc9d5-15700,uri,http://www.w3.org/ns/shacl#path,uri,http://www.w3.org/2004/02/skos/core#notation
5,bnode,anon-genid-b5f6f4f8800e435c9e169bc3b47dc9d5-15700,uri,http://www.w3.org/ns/shacl#datatype,uri,http://www.w3.org/2001/XMLSchema#string
6,bnode,anon-genid-b5f6f4f8800e435c9e169bc3b47dc9d5-15701,uri,http://www.w3.org/ns/shacl#path,uri,http://www.w3.org/2004/02/skos/core#notation
7,bnode,anon-genid-b5f6f4f8800e435c9e169bc3b47dc9d5-15701,uri,http://www.w3.org/ns/shacl#datatype,uri,http://www.w3.org/2001/XMLSchema#string
8,bnode,anon-genid-b17e9d5b34c14ab7a787ee5542937c8b-15700,uri,http://www.w3.org/ns/shacl#path,uri,http://www.w3.org/2004/02/skos/core#notation
9,bnode,anon-genid-b17e9d5b34c14ab7a787ee5542937c8b-15700,uri,http://www.w3.org/ns/shacl#datatype,uri,http://www.w3.org/2001/XMLSchema#string


Questions:

- what are these information used for?

## Findings

### Blocking-Points

- missing /concept/ in URI --> https://register.ld.admin.ch/i14y/legalForm should be https://register.ld.admin.ch/i14y/concept/legalForm --> now corrected
- version.link URIs are wrong: e.g. https://version.link/#Identity should be https://version.link/Identity (all without #) --> now corrected

### Improvements

- a version that has a successor should have a stop date: e.g. https://register.ld.admin.ch/i14y/DV_ST/version/2023.1.0 should have schema:validThrough 2024-05-13 (because version 2024.1.0 has schema:validFrom 2024-05-14), it seems that at least for the DV_ST, there is also no "valid to" on I14Y (see https://www.i14y.admin.ch/de/catalog/concepts/08dbbb66-fb7b-1b35-ad15-80ff9c9bd177/description#section-versions)
- all versions should link via vl:identity to the corresponding identity (not only the newest version), e.g. https://register.ld.admin.ch/i14y/DV_ST/version/2023.1.0 should have vl:identity to <https://register.ld.admin.ch/i14y/DV_ST>, background is, that a version should never have a triple changed or removed (only added)
- I would propose that every concept from I14Y that has the class schema:DefinedTermSet should also have a <https://schema.ld.admin.ch/I14yConcept> class added to better separate it from all the other schema:DefinedTermSet on LINDAS (otherwise, the Query must always be narrowed down on the graph <https://lindas.admin.ch/fso/i14y>)
- usage of predicate https://datacite-metadata-schema.readthedocs.io/en/4.6/appendices/appendix-1/relationType/#haspart seems a little bit odd (it is the only predicate used that comes from this datacite vocabulary), I would propose to use skos:hasPart or xkos:hasPart as alternative
- concerning the /.well-known/void triples: Michael wrote an eMail on 13.06.2025 on this subject, I try to summarize: the single schema:DefinedTermSet do not need a triple schema:includedInDataCatalog <https://register.ld.admin.ch/i14y/.well-known/void> and <https://register.ld.admin.ch/i14y/.well-known/void> should not be used as subject, but the whole i14y dataset should be registered via schema:dataset to <https://register.ld.admin.ch/.well-known/dataset/i14y> in <https://register.ld.admin.ch/.well-known/void>, see the other datasets already present there --> so basically, the .well-known/void mechanism is not used to group all the single concepts into the "i14y dataset", but to register the "i14y dataset" as a whole on register.ld.admin.ch
- versions and identities should have a class vl:Version and vl:Identity respectively --> solved, maybe an error from my side that I did not check correctly